In [1]:
import sims
import numpy as np
import msprime
import pandas as pd

import useful
import seaborn
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload


In [2]:
# Generation time, mutation rate and recomination rate
RR = 1e-8
MU = 1.29e-8 
GEN_time = 29.0 

# Split Times
T_NEAND_migration = 55000 #time of Neanderthal migration into Out_of_africa population
T_NEAND_AMH = 550000 # split time between AMH and Neanderthal
T_OOF_AF = 65700 # Out_of_Africa migration time
T_NEAND_samples = 35000
T_SECOND_NEAND_MIGRATION = 40000

# Effective population size
N_ANC = 18500 # N_e of common  AMH and NEanderthal population 
N_ND = 3400 # N_e of Neanderthal
N_AMH = 23000 # N_e of AMH
N_OOF = 1861 # N_e of Out_of_Africa population
N_AF = 27600 # N_e of Africans
N_EU = 13377 #N_e of Europeans

N_EU_bottleneck = 1080
N_EU_growth = 1450
T_EU_growth = 31900
gr_rate = 0.00202
Portion_admix = 0.1
Portion_admix2 = 0.1

len_sequence = 2e7 # DNA sequence length

n = 50 # number of generated   AF samples
n_neand = 6 #number of generated Neanderthals

rand_sd =1234 #random seed

T = np.array([T_NEAND_migration, T_NEAND_AMH, T_OOF_AF, T_SECOND_NEAND_MIGRATION])/GEN_time
print(T)


[ 1896.55172414 18965.51724138  2265.51724138  1379.31034483]


In [3]:
N_ND = 3400 # N_e of Neanderthal
N_e = np.array([N_ANC, N_ND, N_AMH, N_OOF, N_AF, N_EU])

n_eu=2
ts3400 =sims.history_archaic(GEN_time, len_sequence, RR, MU, N_e, T,  n, rand_sd, n_neand,  
                         T_NEAND_samples/GEN_time, n_eu, N_EU_growth, 
                         T_EU_growth/GEN_time, N_EU_bottleneck, gr_rate, Portion_admix, Portion_admix2)
sims.print_neand_dosages(ts3400)

DemographyDebugger
╠════════════════════════════════════╗
║ Epoch[0]: [0, 1.1e+03) generations ║
╠════════════════════════════════════╝
╟    Populations (total=7 active=5)
║    ┌──────────────────────────────────────────────────────────────────────────┐
║    │       │     start│       end│growth_rate  │ AF │ EU │ NEAND │ OOA │ OOA1 │
║    ├──────────────────────────────────────────────────────────────────────────┤
║    │     AF│   27600.0│   27600.0│ 0           │ 0  │ 0  │   0   │  0  │  0   │
║    │     EU│   13377.0│    1450.0│ 0.00202     │ 0  │ 0  │   0   │  0  │  0   │
║    │  NEAND│    3400.0│    3400.0│ 0           │ 0  │ 0  │   0   │  0  │  0   │
║    │    OOA│    1861.0│    1861.0│ 0           │ 0  │ 0  │   0   │  0  │  0   │
║    │   OOA1│    1861.0│    1861.0│ 0           │ 0  │ 0  │   0   │  0  │  0   │
║    └──────────────────────────────────────────────────────────────────────────┘
╟    Events @ generation 1.1e+03
║    ┌───────────────────────────────────────────────────

In [5]:
ND_true_tracts = [[],[]]
for idx in range(0, n_eu): 
#    print( sims.get_migrating_tracts_ind(ts3400, 'NEAND', idx, T[0]))
    ND_true_tracts[0].append( sims.get_migrating_tracts_ind(ts3400, 'NEAND', idx, T[0]) )
    ND_true_tracts[1].append( sims.get_migrating_tracts_ind(ts3400, 'NEAND', idx, T[3]) )


In [7]:
N_ref_pop=50
N_neanderthal=4
N=3
with open('make.obs.sims.txt', 'w') as f:
    f.write('#POSITIONS\t#REF\t#ALT\tANCESTRAL\t#OUTGROUP\t#ARCHAIC\t#OBSERVATIONS\n')
    for v in ts3400.variants():
        outgroup= str(list(set(v.genotypes[n_eu :( n_eu+N_ref_pop)]))).replace('[','').replace(']','').replace(' ','')
        archaic= str(list(set(v.genotypes[n_eu+n :( n_eu+n+N_neanderthal)]))).replace('[','').replace(']','').replace(' ','')

        obs=''
        for i in v.genotypes[0 :n_eu]:
            obs+=str(i)+' '


        f.write(str(int(v.site.position))+'\t'+str(v.alleles[0])+'\t'+
                str(v.alleles[1]) + '\t'+ str(0)+'\t' + outgroup+'\t'+archaic+'\t'+str(obs)+'\n')    

In [8]:
L=1000

#create bed file
with open('test.bed','w') as f:
    f.write('1\t0\t'+str(int(len_sequence)-1)+'\n')



f_bed='test.bed'
domain=useful.read_bed(f_bed)

n_windows=(domain[-1][1]-domain[0][0])//L + 1
windows_cover=np.ones(n_windows)*0.999

#create archaic covering file. 
CHR=1
with open('arch.covering.chr'+str(CHR)+'.txt','w') as f:
    for j in windows_cover:
        f.write(str(j)+'\n')

print(n_eu)
#create file with sample's names
with open('samples.txt','w') as f:
    for i in range(int(n_eu/2)):
        f.write('eu'+str(i)+'\n')

2


In [17]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed   --HMM_par par.file.txt --EM no --prepared_file make.obs.sims.txt --o out --arch_cover arch.covering.chr1.txt --cut_off 0.9

In [132]:
reload(useful)
HMM_tracts, REAL_tracts=[], []
HMM_ND1, HMM_ND2=useful.read_out('out.archaic.ND1.txt'), useful.read_out('out.archaic.ND2.txt')
for j in range(len(HMM_ND1)):

    nd_hmm=HMM_ND1[j]+HMM_ND2[j]
    if [] in nd_hmm:
        nd_hmm.remove([])
    modern_hmm=useful.tracts_eu(nd_hmm, len_sequence)
    HMM_tracts.append([modern_hmm, HMM_ND1[j], HMM_ND2[j]])
    
    
    nd_real=ND_true_tracts[0][j]+ND_true_tracts[1][j]    
    if [] in nd_real:
        nd_real.remove([])
        
    modern_real=useful.tracts_eu(nd_real, len_sequence)
    REAL_tracts.append([modern_real, ND_true_tracts[0][j], ND_true_tracts[1][j]])    



useful.confusion_mtrx(REAL_tracts[0], HMM_tracts[0])   
df = useful.df_result(  REAL_tracts, HMM_tracts,N_neanderthal, L, N_ref_pop, n_eu, N)

!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!
!!!!!!!


/home/scglab/MyPython/DAIseg2-main/useful.py:252: RuntimeWarning: invalid value encountered in scalar divide
  dd['precision'] = round(conf_matrix[i,i]/sum(conf_matrix[i,:]),7)
/home/scglab/MyPython/DAIseg2-main/useful.py:252: RuntimeWarning: invalid value encountered in scalar divide
  dd['precision'] = round(conf_matrix[i,i]/sum(conf_matrix[i,:]),7)


In [133]:
df

,State,Value,Score,n_eu,n_neand,L,n_ref_pop
0,0,0.998474,precision,0,4,1000,50
1,0,0.919266,recall,0,4,1000,50
2,1,NaN,precision,0,4,1000,50
3,1,0.000000,recall,0,4,1000,50
4,2,0.249725,precision,0,4,1000,50
5,2,0.987973,recall,0,4,1000,50
6,0,0.998756,precision,1,4,1000,50
7,0,0.925538,recall,1,4,1000,50
8,1,NaN,precision,1,4,1000,50
9,1,0.000000,recall,1,4,1000,50
